In [19]:
import os
import sys
from pyspark import SparkContext
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [36]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('test').getOrCreate()
print(spark)
sc = spark.sparkContext
print(sc)

<SparkContext master=local[*] appName=df-project>


In [24]:
#Select Apis And functions
ord = spark.read.load('D:\SpectraMD\Feed\Benchmarks',sep="|",format="csv",header=True)#,schema= ('Year string,State  string,     LOB string,Benchmark_category string,source string, averageName string,5thpercentile string,10thpercentile string,25thPercentile string,33_33thPercentile string,50thPercentile string,66_67thPercentile int,75thPercentile int,90thPercentile int'))
ord.show()
 
ord.summary().show()

#select Apis , fetching cols in multiples ways
ord.select(ord.Year,'Year',"Year",(ord.Year).alias("Year1")).show()

#Applying sql functions on the columns 
from pyspark.sql.functions import lower
ord.select('Year',lower(ord.Year)).show()

#Withcolumns
from pyspark.sql.functions import substring
ord.withColumn('Year1',substring(ord.Year,1,5)).show()

#withColumnRenamed
ord.withColumnRenamed('Year','Year2').show()

#drop 
ord.drop('Year').show()

#Drop Duplicate : drops duplicate rows. optional subset of the columns 

+----------+-----+--------+------------------+------+------------+-------------+--------------+--------------+-----------------+--------------+-----------------+--------------+--------------+--------------+-----------------+-------------+-------------------+
|      Year|State|     LOB|Benchmark_category|source| averageName|5thpercentile|10thpercentile|25thPercentile|33_33thPercentile|50thPercentile|66_67thPercentile|75thPercentile|90thPercentile|95thPercentile|   base_measureid|sub_measureid|   base_measuredesc|
+----------+-----+--------+------------------+------+------------+-------------+--------------+--------------+-----------------+--------------+-----------------+--------------+--------------+--------------+-----------------+-------------+-------------------+
|HEDIS_2021|   FL|Medicaid|Benchmarkcategory1| test1|averageName1|           51|           101|           251|              331|           501|              661|           751|           901|            01|cs_basemeasureid1

In [25]:
#Filter
ord = spark.read.load('D:\SpectraMD\Feed\Benchmarks',sep="|",format="csv",header=True,schema= ('Year string,State  string,     LOB string,Benchmark_category string,source string, averageName string,5thpercentile string,10thpercentile string,25thPercentile string,33_33thPercentile string,50thPercentile string,66_67thPercentile int,75thPercentile int,90thPercentile int'))
ord.show()

#Filter rows using where conditions. use &  and |f
from pyspark.sql.functions import col
ord.where((col('75thPercentile') > 751) & (col('75thPercentile') < 753)).show()


+----------+-----+--------+------------------+------+------------+-------------+--------------+--------------+-----------------+--------------+-----------------+--------------+--------------+
|      Year|State|     LOB|Benchmark_category|source| averageName|5thpercentile|10thpercentile|25thPercentile|33_33thPercentile|50thPercentile|66_67thPercentile|75thPercentile|90thPercentile|
+----------+-----+--------+------------------+------+------------+-------------+--------------+--------------+-----------------+--------------+-----------------+--------------+--------------+
|HEDIS_2021|   FL|Medicaid|Benchmarkcategory1| test1|averageName1|           51|           101|           251|              331|           501|              661|           751|           901|
|HEDIS_2022|   FL|Medicaid|Benchmarkcategory2| test2|averageName2|           52|           102|           252|              332|           502|              662|           752|           902|
|HEDIS_2021|   VA|Medicaid|Benchmarkcate

In [26]:
#union
df1=spark.range(10)
df2=spark.range(5,15)
df1.union(df2).show()

#union Addll ignore the duplicates 
df1.unionAll(df2).show()

#unionByName
df1=spark.createDataFrame(data=[('a',1),('b',2)],schema=('col1 string, col2 int'))
df2=spark.createDataFrame(data=[(2,'b'),(3,'c')],schema=('col2 int, col1 string'))
df1.unionByName(df2).show()

#To remove the duplicates 
df1=spark.createDataFrame(data=[('a',1),('b',2)],schema=('col1 string, col2 int'))
df2=spark.createDataFrame(data=[(2,'b'),(3,'c')],schema=('col2 int, col1 string'))
df1.unionByName(df2).distinct().show()

#intersect -> remove the duplicates
df1=spark.createDataFrame(data=[('a',1),('a',1),('b',2)],schema=('col1 string, col2 int'))
df2=spark.createDataFrame(data=[('a',1),('a',1),('c',2)],schema=('col1 string, col2 int'))
df1.intersect(df2).show()

#intersectAll -> Retain the duplicates 
df1=spark.createDataFrame(data=[('a',1),('a',1),('b',2)],schema=('col1 string, col2 int'))
df2=spark.createDataFrame(data=[('a',1),('a',1),('c',2)],schema=('col1 string, col2 int'))
df1.intersectAll(df2).show()

#exceptAll 
df1=spark.range(1,10)
df2=spark.range(5,10)
df1.exceptAll(df2).show()


+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
+---+

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
+---+

+----+----+
|col1|col2|
+----+----+
|   a|   1|
|   b|   2|
|   b|   2|
|   c|   3|
+----+----+

+----+----+
|col1|col2|
+----+----+
|   a|   1|
|   b|   2|
|   c|   3|
+----+----+

+----+----+
|col1|col2|
+----+----+
|   a|   1|
+----+----+

+----+----+
|col1|col2|
+----+----+
|   a|   1|
|   a|   1|
+----+----+

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
+---+



In [28]:
#JOINS
df1 = spark.createDataFrame(data=[(1,'Robert'),(2,'Kia'),(3,'James')],schema=('empid int, empname string'))
df2 = spark.createDataFrame(data=[(1,'USA'),(4,'INDIA')],schema=('empid int, country string'))

#inner join : Common rows : bydefault join api takes inner 
df1.join(df2,df1.empid == df2.empid).show()

#left join : left + common 
df1.join(df2,df1.empid == df2.empid,'left').show()

#right join : right + common 
df1.join(df2,df1.empid == df2.empid,'right').show()

#full join: right + left
df1.join(df2,df1.empid == df2.empid,'full').show()


+-----+-------+-----+-------+
|empid|empname|empid|country|
+-----+-------+-----+-------+
|    1| Robert|    1|    USA|
+-----+-------+-----+-------+

+-----+-------+-----+-------+
|empid|empname|empid|country|
+-----+-------+-----+-------+
|    1| Robert|    1|    USA|
|    2|    Kia| null|   null|
|    3|  James| null|   null|
+-----+-------+-----+-------+

+-----+-------+-----+-------+
|empid|empname|empid|country|
+-----+-------+-----+-------+
|    1| Robert|    1|    USA|
| null|   null|    4|  INDIA|
+-----+-------+-----+-------+

+-----+-------+-----+-------+
|empid|empname|empid|country|
+-----+-------+-----+-------+
|    1| Robert|    1|    USA|
|    2|    Kia| null|   null|
|    3|  James| null|   null|
| null|   null|    4|  INDIA|
+-----+-------+-----+-------+



In [30]:
#cross : did worked properly need to check 
df1.join(df2,df1.empid == df2.empid,'cross').show()

+-----+-------+-----+-------+
|empid|empname|empid|country|
+-----+-------+-----+-------+
|    1| Robert|    1|    USA|
+-----+-------+-----+-------+



In [31]:
#leftsemi similar to inner join only access the column from left df more efficient
df1 = spark.createDataFrame(data=[(1,'Robert'),(2,'Kia'),(3,'James')],schema=('empid int, empname string'))
df2 = spark.createDataFrame(data=[(1,'USA'),(4,'INDIA')],schema=('empid int, country string'))
df1.join(df2,df1.empid == df2.empid,'leftsemi').show()

+-----+-------+
|empid|empname|
+-----+-------+
|    1| Robert|
+-----+-------+



In [32]:
#Cross join usinf different syntax
df1.crossJoin(df2).show()

+-----+-------+-----+-------+
|empid|empname|empid|country|
+-----+-------+-----+-------+
|    1| Robert|    1|    USA|
|    1| Robert|    4|  INDIA|
|    2|    Kia|    1|    USA|
|    2|    Kia|    4|  INDIA|
|    3|  James|    1|    USA|
|    3|  James|    4|  INDIA|
+-----+-------+-----+-------+



In [10]:
#self Join : join with same table
from pyspark.sql.functions import col
df1 = spark.createDataFrame(data=[(1,'Robert',2),(2,'Kia',3),(3,'James',5)],schema=('empid int, empname string,managerId int'))
df1.alias("emp1").join(df1.alias("emp2"),col("emp1.managerId") == col("emp2.empid"),"inner").select(col("emp1.empId"),col("emp1.empname"),col("emp2.empid").alias("managerId"),col("emp2.empname").alias("managerName")).show()

+-----+-------+---------+-----------+
|empId|empname|managerId|managerName|
+-----+-------+---------+-----------+
|    1| Robert|        2|        Kia|
|    2|    Kia|        3|      James|
+-----+-------+---------+-----------+



In [11]:
#Join on multiple columns 
df1 = spark.createDataFrame(data=[(1,'Robert',3),(2,'Kia',4),(3,'James',5)],schema=('empid int, empname string,deptid int'))
df2 = spark.createDataFrame(data=[(1,'USA',3),(4,'INDIA',4)],schema=('empid int, country string, deptid int'))
df1.join(df2,(df1.empid == df2.empid) & (df1.deptid == df2.deptid) ,'leftsemi').show()

+-----+-------+------+
|empid|empname|deptid|
+-----+-------+------+
|    1| Robert|     3|
+-----+-------+------+



In [12]:
#Aggregations Api
#Avg, max , min
benchmark = spark.read.load('D:\SpectraMD\Feed\Benchmarks',sep="|",format="csv",header=True,schema= ('Year string,State  string,     LOB string,Benchmark_category string,source string, averageName string,5thpercentile string,10thpercentile string,25thPercentile string,33_33thPercentile string,50thPercentile string,66_67thPercentile int,75thPercentile int,90thPercentile int'))
benchmark.show()

from pyspark.sql.functions import *
benchmark.select(avg(col("33_33thPercentile"))).show()

#max
benchmark.select(max(col("33_33thPercentile"))).show()

#min
benchmark.select(min(col("33_33thPercentile"))).show()

#sum sum of all the value .sumDistinct sum of all the distinct values
benchmark.select(sum(col("33_33thPercentile")),sumDistinct(col("33_33thPercentile"))).show()

#count CountDistinct
benchmark.select(count(col("33_33thPercentile")),countDistinct(col("33_33thPercentile"))).show()




+----------+-----+--------+------------------+------+------------+-------------+--------------+--------------+-----------------+--------------+-----------------+--------------+--------------+
|      Year|State|     LOB|Benchmark_category|source| averageName|5thpercentile|10thpercentile|25thPercentile|33_33thPercentile|50thPercentile|66_67thPercentile|75thPercentile|90thPercentile|
+----------+-----+--------+------------------+------+------------+-------------+--------------+--------------+-----------------+--------------+-----------------+--------------+--------------+
|HEDIS_2021|   FL|Medicaid|Benchmarkcategory1| test1|averageName1|           51|           101|           251|              331|           501|              661|           751|           901|
|HEDIS_2022|   FL|Medicaid|Benchmarkcategory2| test2|averageName2|           52|           102|           252|              332|           502|              662|           752|           902|
|HEDIS_2021|   VA|Medicaid|Benchmarkcate

D:\SpectraMD\spark-3.4.1-bin-hadoop3\python\pyspark\sql\functions.py:752: FutureWarning: Deprecated in 3.2, use sum_distinct instead.
  warnings.warn("Deprecated in 3.2, use sum_distinct instead.", FutureWarning)


+----------------------+-------------------------------+
|sum(33_33thPercentile)|sum(DISTINCT 33_33thPercentile)|
+----------------------+-------------------------------+
|                1659.0|                          996.0|
+----------------------+-------------------------------+

+------------------------+---------------------------------+
|count(33_33thPercentile)|count(DISTINCT 33_33thPercentile)|
+------------------------+---------------------------------+
|                       5|                                3|
+------------------------+---------------------------------+



In [13]:
benchmark = spark.read.load('D:\SpectraMD\Feed\Benchmarks',sep="|",format="csv",header=True,schema= ('Year string,State  string,     LOB string,Benchmark_category string,source string, averageName string,5thpercentile string,10thpercentile string,25thPercentile string,33_33thPercentile string,50thPercentile string,66_67thPercentile int,75thPercentile int,90thPercentile int'))
benchmark.show()
from pyspark.sql.functions import *
#first get the first record value , last get the lat value of the record value
benchmark.select(first(col("33_33thPercentile")),last(col("33_33thPercentile"))).show()

#Collect_list(contain all the values) & collect_set(collect unique values)
benchmark.select(collect_list(col("33_33thPercentile")),collect_set(col("33_33thPercentile"))).show()

#

+----------+-----+--------+------------------+------+------------+-------------+--------------+--------------+-----------------+--------------+-----------------+--------------+--------------+
|      Year|State|     LOB|Benchmark_category|source| averageName|5thpercentile|10thpercentile|25thPercentile|33_33thPercentile|50thPercentile|66_67thPercentile|75thPercentile|90thPercentile|
+----------+-----+--------+------------------+------+------------+-------------+--------------+--------------+-----------------+--------------+-----------------+--------------+--------------+
|HEDIS_2021|   FL|Medicaid|Benchmarkcategory1| test1|averageName1|           51|           101|           251|              331|           501|              661|           751|           901|
|HEDIS_2022|   FL|Medicaid|Benchmarkcategory2| test2|averageName2|           52|           102|           252|              332|           502|              662|           752|           902|
|HEDIS_2021|   VA|Medicaid|Benchmarkcate

In [14]:
#GROUP BY
data = [('James','Sales','NY',9000,34),
        ('Alicia','Sales','NY',8600,56),
        ('Robert','Sales','CA',8100,30),
        ('Lina','Finance','CA',9000,24),
        ('Deja','Finance','CA',9900,40),
        ('Sugie','Finance','NY',8300,36),
        ('Ram','Finance','NY',7900,53),
        ('Kyle','Marketing','CA',8000,25),
        ('Reid','Marketing','NY',9100,50)]
schema = ["empname","dept","state","salary","age"]
df = spark.createDataFrame(data,schema)
df.show()

#avg
df.groupBy(df.dept).avg("salary").show()

#sum
df.groupBy(df.dept).sum("salary").show()

#max
df.groupBy(df.dept).max("salary").show()

#multiples columns 
df.groupBy(df.dept,df.state).min("salary","age").show()


+-------+---------+-----+------+---+
|empname|     dept|state|salary|age|
+-------+---------+-----+------+---+
|  James|    Sales|   NY|  9000| 34|
| Alicia|    Sales|   NY|  8600| 56|
| Robert|    Sales|   CA|  8100| 30|
|   Lina|  Finance|   CA|  9000| 24|
|   Deja|  Finance|   CA|  9900| 40|
|  Sugie|  Finance|   NY|  8300| 36|
|    Ram|  Finance|   NY|  7900| 53|
|   Kyle|Marketing|   CA|  8000| 25|
|   Reid|Marketing|   NY|  9100| 50|
+-------+---------+-----+------+---+

+---------+-----------------+
|     dept|      avg(salary)|
+---------+-----------------+
|    Sales|8566.666666666666|
|  Finance|           8775.0|
|Marketing|           8550.0|
+---------+-----------------+

+---------+-----------+
|     dept|sum(salary)|
+---------+-----------+
|    Sales|      25700|
|  Finance|      35100|
|Marketing|      17100|
+---------+-----------+

+---------+-----------+
|     dept|max(salary)|
+---------+-----------+
|    Sales|       9000|
|  Finance|       9900|
|Marketing|       

In [15]:
#Agg -> used for applying multiple agg function
df.groupBy(df.dept).agg(min("salary"),max("salary"),avg("salary")).show()

#Note : filter has to applied before the aggregation if you want apply after the agg then on ly
# apply on the column used in agg function 


+---------+-----------+-----------+-----------------+
|     dept|min(salary)|max(salary)|      avg(salary)|
+---------+-----------+-----------+-----------------+
|    Sales|       8100|       9000|8566.666666666666|
|  Finance|       7900|       9900|           8775.0|
|Marketing|       8000|       9100|           8550.0|
+---------+-----------+-----------+-----------------+



In [33]:
#pivot : transfer rows in to columns 
df = df.select(df.dept,df.state,df.salary)
df1 = df.groupBy(df.dept,df.state).sum("salary").alias("salary")
df1.show()

#applying pivot function
df1.groupBy(df1.dept).pivot("state").sum("sum(salary)").show()


+---------+-----+-----------+
|     dept|state|sum(salary)|
+---------+-----+-----------+
|    Sales|   NY|      17600|
|    Sales|   CA|       8100|
|  Finance|   CA|      18900|
|  Finance|   NY|      16200|
|Marketing|   NY|       9100|
|Marketing|   CA|       8000|
+---------+-----+-----------+

+---------+-----+-----+
|     dept|   CA|   NY|
+---------+-----+-----+
|    Sales| 8100|17600|
|  Finance|18900|16200|
|Marketing| 8000| 9100|
+---------+-----+-----+



In [34]:
#Windows function
from pyspark.sql.window import *
from pyspark.sql.functions import *
data = [('James','Sales','NY',9000,34),
        ('Alicia','Sales','NY',8600,56),
        ('Robert','Sales','CA',8100,30),
        ('Lina','Finance','CA',9000,24),
        ('Deja','Finance','CA',9900,40),
        ('Sugie','Finance','NY',8300,36),
        ('Ram','Finance','NY',7900,53),
        ('Kyle','Marketing','CA',8000,25),
        ('Reid','Marketing','NY',9100,50)]
schema = ["empname","dept","state","salary","age"]
df = spark.createDataFrame(data,schema)
#df.show()
spec = Window.partitionBy("dept").orderBy(df.salary.desc())
df1 = df.select(df.dept,df.salary)
#df1.show()

#df.select(df.dept,df.salary).withColumn("row_number_rank",row_number().over(spec)).show()





+---------+------+---------------+
|     dept|salary|row_number_rank|
+---------+------+---------------+
|  Finance|  9900|              1|
|  Finance|  9000|              2|
|  Finance|  8300|              3|
|  Finance|  7900|              4|
|Marketing|  9100|              1|
|Marketing|  8000|              2|
|    Sales|  9000|              1|
|    Sales|  8600|              2|
|    Sales|  8100|              3|
+---------+------+---------------+

